In [ ]:
# This is the basic proportional rate diffusion model from Palmer, Huk and Shadlen '05. 
# The overall fits are good for subject accuracies but mRT fits are not very good. 
# More specifically, the lowering of mRTs observed at low distance and high coherence 
# is not captured by the model.

# I am not imposing any constraints on the parameters (+ve or -ve) except to say that x is positive.

In [ ]:
## These are the pre-modelling steps where behavioral data is extracted from files.

# Load required libraries
% matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import factorial as fact

# List subjects to fit
subs = ['Sub01', 'Sub02', 'Sub04', 'Sub05', 'Sub06', 'Sub08', 'Sub10', 'Sub11', 'Sub13']
nSub = len(subs)

# Initialize arrays to hold PC, mean and SD of RT, and # trials 
# for each coherence-distanct combination
# These values are obtained from the .csv files
pCs = np.zeros((20, nSub))
mRTs = np.zeros((20, nSub))
sdRTs = np.zeros((20, nSub))
Ns = np.zeros((20, nSub))

# Initialise a variable to hold # correct trials
# This will be computed from Ns and pCs
Rs = np.zeros((20, nSub))

# Extract behavioral data (PC, mean and SD of RT, # trials) from csv files
for si in range(nSub):
    csvFile = '../Data/Behavior/' + subs[si] + '_behavData.csv'
    behavData = pd.read_csv(csvFile, header=None)
    
    # Split the file in PC, mean RT and SD RT
    # Flatten each subject's values for ease of programming
    pCs[:,si] = np.array(behavData[0:4]).flatten()
    mRTs[:,si] = np.array(behavData[4:8]).flatten()
    sdRTs[:,si] = np.array(behavData[8:12]).flatten()
    Ns[:,si] = np.array(behavData[12:]).flatten()
    Rs[:,si] = np.round(Ns[:,si] * pCs[:,si])

In [ ]:
## Model definition

# Parameters to fit:
# A: Boundary
# x: Stimulus strength
# k: Proportionality constant (Stim. str. = kx)
# tR: Residual time

# Equations to fit for each stimulus strength:
# pC = 1 / (1+exp(-2*A*k*abs(x)))
# mRT = A / (k*x) * tanh(A*k*x) + tR

# We can get approximate values for A, k and tR from Palmer et. al. '05
# Ranges of parameters to start with:
# A: 0.5 - 1
# k: 5 - 40
# tR: 0.25 - 0.5 (in seconds)
# x: 0 - 1

# To identify best fit, calculate the likelihood of predicted pC and mRT and find the maximum likelihood.

# Likelihood of pC follows a binomial distribution
# Lp = n! / (r!(n-r)! * pC(x)^r * (1-pC(x))^(n-r), where
# n = # trials, r = # required correct

# Likelihood of mRT follows a Gaussian distribution
# Lrt = 1 / (SDrt * (sqrt(2*pi))) * e^-((mRT(x) - oRT(x)) / SDrt)^2 * 1/2, where
# oRT = observed mRT, mRT = predicted mRT, SDrt = SD of predicted mRT

# VARrt = VARtd + VARtr, where
# VARtd = variance in decision time, VARtr = variance in residual time. Thus,
# VARrt = (A * tanh(A*k*x) - A*k*x * sech(A*k*x)) / (k*x)^3 + (0.1 * tR)^2

# Final fit measure is the log likelihood, which is the sum of the likelihoods of accuracy and mean RT, 
# over all combinations of coherence and distance
# Lprt = sigma(x)(ln(Lp(s)) + ln(Lrt(x)))

# The first pass of the model will be to estimate values of x without any assumptions about stimulus relationtips.
# The stopping point will be the point of least error.

In [ ]:
## First run of the model. 

# Initialize A, k and tR parameters
A, dA = np.linspace(-5, 5, 10, retstep=True)
# k, dk = np.linspace(0.3, 2, 20, retstep=True)
k, dk = np.linspace(-5, 5, 10, retstep=True)
tR, dtR = np.linspace(-5, 5, 10, retstep=True)

As, ks, tRs = np.meshgrid(A, k, tR)
As = As.flatten()
ks = ks.flatten()
tRs = tRs.flatten()

# This is the overall number of permutations of A, k and tR being performed
nPar = len(As)

# Initialize stimulus strength parameter
# x, dx = np.linspace(0.1, 10, 200, retstep=True)
x, dx = np.linspace(0.01, 5, 200, retstep=True)

# Initialize arrays that hold predicted accuracies and RTs
epc = np.ones((nPar, len(x), 20, nSub)) * -9
ert = np.ones((nPar, len(x), 20, nSub)) * -9
sdrt = np.ones((nPar, len(x), 20, nSub)) * -9

## Initialize the array that holds the
# individual likelihood values
lpc = np.zeros((nPar, len(x), 20, nSub))
lrt = np.zeros((nPar, len(x), 20, nSub))

In [ ]:
# From the set of parameters, calculate the (expected) predicted PC and RT
# and then find the likelihood that these estimates match the observed PC and RT
for si in range(nSub):
    for pi in xrange(nPar):
        for cdi in range(20):
            # Calculate expected accuracy for each coherence-distance combination
            epc[pi,:,cdi,si] = 1 / (1 + np.exp(-2 * As[pi] * ks[pi] * abs(x)))
            # And the likelihood of this accuracy
            lpc[pi,:,cdi,si] = fact(Ns[cdi,si]) / (fact(Rs[cdi,si]) * fact(Ns[cdi,si]-Rs[cdi,si])) * \
                                (epc[pi,:,cdi,si] ** Rs[cdi,si]) * \
                                ((1 - epc[pi,:,cdi,si]) ** (Ns[cdi,si] - Rs[cdi,si]))
            
            # Calculate expected mean RT for each coherence-distance combination
            ert[pi,:,cdi,si] = As[pi] / (ks[pi] * x) * np.tanh(As[pi] * ks[pi] * x) + tRs[pi] 
            # And standard error of the mean
            sdrt[pi,:,cdi,si] = np.sqrt(((As[pi] * np.tanh(As[pi] * ks[pi] * x) - \
                    As[pi] * ks[pi] * x * (1/np.cosh(np.square(As[pi] * ks[pi] * x)))) / \
                    (ks[pi] * x) ** 3 + np.square(0.1 * tRs[pi])) / Ns[cdi,si])
            # And the likelihood of observing that RT
            lrt[pi,:,cdi,si] = 1 / (sdrt[pi,:,cdi,si] * np.sqrt(2 * np.pi)) * \
                                np.e ** (-1/2 * np.square((ert[pi,:,cdi,si] - mRTs[cdi,si]) / sdrt[pi,:,cdi,si]))

In [ ]:
# See if there are any weird PC, mRT or sdrt values generated in the parameter space
print(np.sum(np.isinf(epc)))
print(np.sum(np.isnan(epc)))
print(np.sum(epc < 0))
print(np.sum(np.isinf(ert)))
print(np.sum(np.isnan(ert)))
print(np.sum(ert < 0))
print(np.sum(np.isinf(sdrt)))
print(np.sum(np.isnan(sdrt)))
print(np.sum(sdrt < 0))

In [ ]:
# This figure takes a long time to plot because there is a lot of data to 
# flatten and then plot. The gist is that: 
# - lpc ranges from 0 to 1 for all sujbects
# - lrt ranges from 0 to 60 for all subjects

plt.figure()
for si in range(nSub):
    plt.subplot(3,3,si+1)
    plt.plot(lpc[...,si].flatten(), '.')
    
plt.figure()
for si in range(nSub):
    plt.subplot(3,3,si+1)
    plt.plot(lrt[...,si].flatten(), '.')

In [ ]:
# Find indices where sdrt is NaN (or, variance is negative)
tmpId = np.where(np.isnan(sdrt))[0]
sdIdx = np.unravel_index(tmpId, sdrt.shape)

# Find the fraction of trials that have NaN sdrt - 0.0134
print(len(sdIdx[0])/np.double(np.prod(sdrt.shape)))

# First see if it's any specific subjects that show this effect - no
print(np.unique(sdIdx[3]))
for i in range(nSub):
    print(np.sum(sdIdx[3] == i))

# Any specific CD combinations - no
print(np.unique(sdIdx[2]))
for i in range(20):
    print(np.sum(sdIdx[2] == i))

# Any specific x's - smaller ones
print(np.unique(sdIdx[1]))
for i in range(len(x)):
    print(np.sum(sdIdx[1] == i))

# Any specific A-k-tR combinations - first one
print(np.unique(sdIdx[0]))

In [ ]:
# First find the best values of x for each CD combination.
# Once that is done, calculate the likelihood of each A-k-tR combination
# at the best x values.

# NOTE: After careful consideration, I have determined that normalizing the 
# likelihood values does not give a better fit.

bestXcd = np.ones((nPar, 20, nSub), dtype = np.int) * -9
totLL = np.zeros((nPar, nSub))

for si in range(nSub):
    for pi in range(nPar):
        # At each A-k-tR combination, find the index of x value 
        # that maximizes likelihood for each CD combination
        bestXcd[pi,:,si] = np.argmax(lpc[pi,:,:,si] * lrt[pi,:,:,si], 0)
        
        for cdi in range(20):
            # Sum over all CDs to obtain overall likelihood for the 
            # give A-k-tR combination
            totLLpc[pi,si] += lpc[pi,bestXcd[pi,cdi,si],cdi,si]
            totLLrt[pi,si] += lrt[pi,bestXcd[pi,cdi,si],cdi,si]
            totLL[pi,si] += np.log(lpc[pi,bestXcd[pi,cdi,si],cdi,si]) + \
                            np.log(lrt[pi,bestXcd[pi,cdi,si],cdi,si])

In [ ]:
# Find the parameters for which likelihood is maximum
# There are some NaN values in the likelihood matrix so exclude those
maxParId = np.zeros(nSub, dtype=np.int)

# Record the values of best fit x for each CD combination
bestx = np.zeros((20, nSub), dtype = np.int)

for si in range(nSub):
    maxParId[si] = np.where(totLL[:,si] == np.nanmax(totLL[:,si]))[0]
    bestx[:,si] = bestXcd[maxParId[si],:,si]
    # print(np.where(maxLLx[:,si] == np.nanmax(maxLLx[:,si]))[0])

In [ ]:
for si in range(nSub):
    print(maxParId[si], round(totLL[maxParId[si],si],ndigits=3))

In [ ]:
subXs = np.zeros((20, nSub))
# Print the parameter values predicting maximum likelihoods
# for the different coherence-distance combinations
for si in range(nSub):
    print(round(As[maxParId[si]],ndigits=3), round(ks[maxParId[si]],ndigits=3), round(tRs[maxParId[si]],ndigits=3))
    print(np.reshape(x[bestx[:,si]].round(decimals=2),(4,5)))
    #subXs[:,si] = x[bestx[:,si]].round(decimals=2)
    
#np.savetxt('SubXs_all.csv', subXs, fmt = '%.2f', delimiter = ',')

In [ ]:
# Plot the observed (behavior) and expected (model) PC and RT
for si in range(nSub):
    temp = []
    ymin = min(pCs[:,si])
    ymax = max(pCs[:,si])
    for cdi in range(20):
        temp.append(epc[maxParId[si], bestx[cdi,si], cdi, si])
    if min(temp) < ymin:
        ymin = min(temp)
    if max(temp) > ymax:
        ymax = max(temp)
    
    plt.figure()
    plt.subplot(221)
    plt.title(subs[si] + ' Behavior')
    plt.plot(np.reshape(pCs[:,si],(4,5)),'o-')
    plt.ylim((ymin, ymax))
    
    plt.subplot(222)
    plt.title('Model1')
    plt.plot(np.reshape(temp,(4,5)),'o-')
    plt.ylim((ymin,ymax))
    
    temp = []
    ymin = min(mRTs[:,si])
    ymax = max(mRTs[:,si])
    for cdi in range(20):
        temp.append(ert[maxParId[si], bestx[cdi,si], cdi, si])
    if min(temp) < ymin:
        ymin = min(temp)
    if max(temp) > ymax:
        ymax = max(temp)
    
    plt.subplot(223)
    plt.plot(np.reshape(mRTs[:,si],(4,5)),'o-')
    plt.ylim((ymin,ymax))
    plt.xlabel('Coherence')
    
    plt.subplot(224)
    plt.plot(np.reshape(temp,(4,5)),'o-')
    plt.ylim((ymin,ymax))
    plt.xlabel('Coherence')

In [ ]:
minA = np.zeros(nSub)
maxA = np.zeros(nSub)
mink = np.zeros(nSub)
maxk = np.zeros(nSub)
mintR = np.zeros(nSub)
maxtR = np.zeros(nSub)

nd = 3 # Number of deltas before and after best fit value

for si in range(nSub):
    # Set up A, k and tR parameters for the next round of simulations
    # Use the bestx values from the first run, don't fit for x again
    
    # First set the range of all variables. 
    # minVar = bestValue - dVar*nd : bestValue + dVar*nd
    # If the new minimum is <= 0, then set it to the old minimum.
    minA[si] = As[maxParId[si]] - nd * dA
    maxA[si] = As[maxParId[si]] + nd * dA
    # if minA[si] < 0:
    #    minA[si] = A[0]
    
    mink[si] = ks[maxParId[si]] - nd * dk
    maxk[si] = ks[maxParId[si]] + nd * dk
    # if mink[si] < 0:
    #    mink[si] = k[0]
        
    mintR[si] = tRs[maxParId[si]] - nd * dtR
    maxtR[si] = tRs[maxParId[si]] + nd * dtR
    # if mintR[si] < 0:
    #    mintR[si] = tR[0]

In [ ]:
print(minA.round(decimals=3))
print(maxA.round(decimals=3))
print(mink.round(decimals=3))
print(maxk.round(decimals=3))
print(mintR.round(decimals=3))
print(maxtR.round(decimals=3))

In [ ]:
nStep = 9 # Number of values tested, per parameter

A = np.zeros((nStep, nSub))
dA = np.zeros(nSub)
k = np.zeros((nStep, nSub))
dk = np.zeros(nSub)
tR = np.zeros((nStep, nSub))
dtR = np.zeros(nSub)

# This is the overall number of permutations of A, k and tR being performed
nPar = nStep ** 3

As = np.zeros((nPar, nSub))
ks = np.zeros((nPar, nSub))
tRs = np.zeros((nPar, nSub))

for si in range(nSub):
    A[:,si], dA[si] = np.linspace(minA[si], maxA[si], nStep, retstep=True)
    k[:,si], dk[si] = np.linspace(mink[si], maxk[si], nStep, retstep=True)
    tR[:,si], dtR[si] = np.linspace(mintR[si], maxtR[si], nStep, retstep=True)

    tempAs, tempks, temptRs = np.meshgrid(A[:,si], k[:,si], tR[:,si])
    As[:,si] = tempAs.flatten()
    ks[:,si] = tempks.flatten()
    tRs[:,si] = temptRs.flatten()
    
    # Initialize arrays that hold predicted accuracies and RTs
    epc = np.ones((nPar, 20, 9)) * -9
    ert = np.ones((nPar, 20, 9)) * -9
    sdrt = np.ones((nPar, 20, 9)) * -9

    ## Initialize the array that holds the
    # individual likelihood values
    lpc = np.zeros((nPar, 20, 9))
    lrt = np.zeros((nPar, 20, 9))

In [ ]:
# From the set of parameters, calculate the (expected) predicted PC and RT
# and then find the likelihood that these estimates match the observed PC and RT
for si in range(nSub):
    for pi in xrange(nPar):
        for cdi in range(20):
            # Calculate expected accuracy for each coherence-distance combination
            epc[pi,cdi,si] = 1 / (1 + np.exp(-2 * As[pi,si] * ks[pi,si] * abs(x[bestx[cdi,si]])))

            # Calculate likelihood of accuracy for this CD combination
            lpc[pi,cdi,si] = fact(Ns[cdi,si]) / (fact(Rs[cdi,si]) * fact(Ns[cdi,si]-Rs[cdi,si])) * \
                                (epc[pi,cdi,si] ** Rs[cdi,si]) * \
                                ((1 - epc[pi,cdi,si]) ** (Ns[cdi,si] - Rs[cdi,si]))
            
            # Calculate expected mean RT for each coherence-distance combination
            ert[pi,cdi,si] = As[pi,si] / (ks[pi,si] * x[bestx[cdi,si]]) * \
                                np.tanh(As[pi,si] * ks[pi,si] * x[bestx[cdi,si]]) + tRs[pi,si] 
            # Calculate SD of mean RT 
            sdrt[pi,cdi,si] = np.sqrt(((As[pi,si] * np.tanh(As[pi,si] * ks[pi,si] * x[bestx[cdi,si]]) - \
                    As[pi,si] * ks[pi,si] * x[bestx[cdi,si]] * \
                    (1/np.cosh(np.square(As[pi,si] * ks[pi,si] * x[bestx[cdi,si]])))) / \
                    (ks[pi,si] * x[bestx[cdi,si]]) ** 3 + np.square(0.1 * tRs[pi,si])) / Ns[cdi,si])
            # Calculate likelihood of mean RT for this CD combination
            lrt[pi,cdi,si] = 1 / (sdrt[pi,cdi,si] * np.sqrt(2 * np.pi)) * \
                                np.e ** (-1/2 * np.square((ert[pi,cdi,si] - mRTs[cdi,si]) / sdrt[pi,cdi,si]))

In [ ]:
#maxLLcd = np.ones((nPar, 20, nSub)) * -9
totLL = np.zeros((nPar, nSub))

for si in range(nSub):
    for pi in range(nPar):
        # maxLLcd[pi,:,si] = np.argmax(lpc[pi,:,:,si]+lrt[pi,:,:,si],0)
        for cdi in range(20):
            #totLL[pi,si] += np.log(lpc[pi,cdi,si]) + \
            #                    np.log(lrt[pi,cdi,si])
            totLL[pi,si] += np.log(lpc[pi,cdi,si]) + \
                            np.log(lrt[pi,cdi,si])

In [ ]:
for si in range(nSub):
    plt.subplot(3,3,si+1)
    plt.plot(totLL[:,si],'.')

In [ ]:
# Find the parameters for which likelihood is maximum
# There are some NaN values in the likelihood matrix so exclude those
maxParId = np.zeros(nSub)

for si in range(nSub):
    maxParId[si] = np.where(totLL[:,si] == np.nanmax(totLL[:,si]))[0]
    # print(np.where(maxLLx[:,si] == np.nanmax(maxLLx[:,si]))[0])

In [ ]:
for si in range(nSub):
    print(totLL[maxParId[si],si])

In [ ]:
# Print the parameter values predicting maximum likelihoods
# for the different coherence-distance combinations
for si in range(nSub):
    print(round(As[maxParId[si],si],ndigits=3), round(ks[maxParId[si],si],ndigits=3), \
          round(tRs[maxParId[si],si],ndigits=3))
    # print(np.reshape(maxLLcd[maxParId[si],:,si],(4,5)))

In [ ]:
# Plot the observed (behavior) and expected (model) PC and RT
for si in range(nSub):
    temp = []
    ymin = min(pCs[:,si])
    ymax = max(pCs[:,si])
    for cdi in range(20):
        temp.append(epc[maxParId[si],cdi,si])
    if min(temp) < ymin:
        ymin = min(temp)
    if max(temp) > ymax:
        ymax = max(temp)
    
    plt.figure()
    plt.subplot(221)
    plt.title(subs[si] + ' Behavior')
    plt.plot(np.reshape(pCs[:,si],(4,5)),'o-')
    plt.ylim((ymin, ymax))
    
    plt.subplot(222)
    plt.title('Model')
    plt.plot(np.reshape(temp,(4,5)),'o-')
    plt.ylim((ymin,ymax))

    temp = []
    ymin = min(mRTs[:,si])
    ymax = max(mRTs[:,si])
    for cdi in range(20):
        temp.append(ert[maxParId[si],cdi,si])
    if min(temp) < ymin:
        ymin = min(temp)
    if max(temp) > ymax:
        ymax = max(temp)
    
    plt.subplot(223)
    plt.plot(np.reshape(mRTs[:,si],(4,5)),'o-')
    plt.ylim((ymin,ymax))
    plt.xlabel('Coherence')
    
    plt.subplot(224)
    plt.plot(np.reshape(temp,(4,5)),'o-')
    plt.ylim((ymin,ymax))
    plt.xlabel('Coherence')

In [ ]:
minA = np.zeros(nSub)
maxA = np.zeros(nSub)
mink = np.zeros(nSub)
maxk = np.zeros(nSub)
mintR = np.zeros(nSub)
maxtR = np.zeros(nSub)

for si in range(nSub):
    # Set up A, k and tR parameters for the next round of simulations
    # Use the bestx values from the first run, don't fit for x again
    
    # First set the range of all variables. 
    # minVar = bestValue - dVar*nd : bestValue + dVar*nd
    # If the new minimum is <= 0, then set it to the old minimum.
    minA[si] = As[maxParId[si],si] - nd * dA[si]
    maxA[si] = As[maxParId[si],si] + nd * dA[si]
    # if minA[si] < 0:
    #    minA[si] = A[0,si]
    
    mink[si] = ks[maxParId[si],si] - nd * dk[si]
    maxk[si] = ks[maxParId[si],si] + nd * dk[si]
    # if mink[si] < 0:
    #    mink[si] = k[0,si]
        
    mintR[si] = tRs[maxParId[si],si] - nd * dtR[si]
    maxtR[si] = tRs[maxParId[si],si] + nd * dtR[si]
    # if mintR[si] < 0:
    #    mintR[si] = tR[0,si]

print(minA.round(decimals=3))
print(maxA.round(decimals=3))
print(mink.round(decimals=3))
print(maxk.round(decimals=3))
print(mintR.round(decimals=3))
print(maxtR.round(decimals=3))

A = np.zeros((nStep, nSub))
dA = np.zeros(nSub)
k = np.zeros((nStep, nSub))
dk = np.zeros(nSub)
tR = np.zeros((nStep, nSub))
dtR = np.zeros(nSub)

As = np.zeros((nPar, nSub))
ks = np.zeros((nPar, nSub))
tRs = np.zeros((nPar, nSub))

for si in range(nSub):
    A[:,si], dA[si] = np.linspace(minA[si], maxA[si], nStep, retstep=True)
    k[:,si], dk[si] = np.linspace(mink[si], maxk[si], nStep, retstep=True)
    tR[:,si], dtR[si] = np.linspace(mintR[si], maxtR[si], nStep, retstep=True)

    tempAs, tempks, temptRs = np.meshgrid(A[:,si], k[:,si], tR[:,si])
    As[:,si] = tempAs.flatten()
    ks[:,si] = tempks.flatten()
    tRs[:,si] = temptRs.flatten()

    # Initialize arrays that hold predicted accuracies and RTs
    epc = np.ones((nPar, 20, 9)) * -9
    ert = np.ones((nPar, 20, 9)) * -9
    sdrt = np.ones((nPar, 20, 9)) * -9

    ## Initialize the array that holds the
    # individual likelihood values
    lpc = np.zeros((nPar, 20, 9))
    lrt = np.zeros((nPar, 20, 9))
    
# From the set of parameters, calculate the (expected) predicted PC and RT
# and then find the likelihood that these estimates match the observed PC and RT
for si in range(nSub):
    for pi in xrange(nPar):
        for cdi in range(20):
            # Calculate expected accuracy for each coherence-distance combination
            epc[pi,cdi,si] = 1 / (1 + np.exp(-2 * As[pi,si] * ks[pi,si] * abs(x[bestx[cdi,si]])))

            # Calculate likelihood of accuracy for this CD combination
            lpc[pi,cdi,si] = fact(Ns[cdi,si]) / (fact(Rs[cdi,si]) * fact(Ns[cdi,si]-Rs[cdi,si])) * \
                                (epc[pi,cdi,si] ** Rs[cdi,si]) * \
                                ((1 - epc[pi,cdi,si]) ** (Ns[cdi,si] - Rs[cdi,si]))
            
            # Calculate expected mean RT for each coherence-distance combination
            ert[pi,cdi,si] = As[pi,si] / (ks[pi,si] * x[bestx[cdi,si]]) * \
                                np.tanh(As[pi,si] * ks[pi,si] * x[bestx[cdi,si]]) + tRs[pi,si] 
            # Calculate SD of mean RT 
            sdrt[pi,cdi,si] = np.sqrt(((As[pi,si] * np.tanh(As[pi,si] * ks[pi,si] * x[bestx[cdi,si]]) - \
                    As[pi,si] * ks[pi,si] * x[bestx[cdi,si]] * \
                    (1/np.cosh(np.square(As[pi,si] * ks[pi,si] * x[bestx[cdi,si]])))) / \
                    (ks[pi,si] * x[bestx[cdi,si]]) ** 3 + np.square(0.1 * tRs[pi,si])) / Ns[cdi,si])
            # Calculate likelihood of mean RT for this CD combination
            lrt[pi,cdi,si] = 1 / (sdrt[pi,cdi,si] * np.sqrt(2 * np.pi)) * \
                                np.e ** (-1/2 * np.square((ert[pi,cdi,si] - mRTs[cdi,si]) / sdrt[pi,cdi,si]))
                
#maxLLcd = np.ones((nPar, 20, nSub)) * -9
totLL = np.zeros((nPar, nSub))

for si in range(nSub):
    for pi in range(nPar):
        # maxLLcd[pi,:,si] = np.argmax(lpc[pi,:,:,si]+lrt[pi,:,:,si],0)
        for cdi in range(20):
            totLL[pi,si] += np.log(lpc[pi,cdi,si]) + \
                                np.log(lrt[pi,cdi,si])
                
for si in range(nSub):
    plt.subplot(3,3,si+1)
    plt.plot(totLL[:,si],'.')
    
# Find the parameters for which likelihood is maximum
# There are some NaN values in the likelihood matrix so exclude those
maxParId = np.zeros(nSub)

for si in range(nSub):
    maxParId[si] = np.where(totLL[:,si] == np.nanmax(totLL[:,si]))[0]
    # print(np.where(maxLLx[:,si] == np.nanmax(maxLLx[:,si]))[0])
    
for si in range(nSub):
    print(totLL[maxParId[si],si])
    
# Print the parameter values predicting maximum likelihoods
# for the different coherence-distance combinations
for si in range(nSub):
    print(round(As[maxParId[si],si],ndigits=3), round(ks[maxParId[si],si],ndigits=3), \
          round(tRs[maxParId[si],si],ndigits=3))
    # print(np.reshape(maxLLcd[maxParId[si],:,si],(4,5)))
    
# Plot the observed (behavior) and expected (model) PC and RT
for si in range(nSub):
    temp = []
    ymin = min(pCs[:,si])
    ymax = max(pCs[:,si])
    for cdi in range(20):
        temp.append(epc[maxParId[si],cdi,si])
    if min(temp) < ymin:
        ymin = min(temp)
    if max(temp) > ymax:
        ymax = max(temp)
    
    plt.figure()
    plt.subplot(221)
    plt.title(subs[si] + ' Behavior')
    plt.plot(np.reshape(pCs[:,si],(4,5)),'o-')
    plt.ylim((ymin, ymax))
    
    plt.subplot(222)
    plt.title('Model')
    plt.plot(np.reshape(temp,(4,5)),'o-')
    plt.ylim((ymin,ymax))

    temp = []
    ymin = min(mRTs[:,si])
    ymax = max(mRTs[:,si])
    for cdi in range(20):
        temp.append(ert[maxParId[si],cdi,si])
    if min(temp) < ymin:
        ymin = min(temp)
    if max(temp) > ymax:
        ymax = max(temp)
    
    plt.subplot(223)
    plt.plot(np.reshape(mRTs[:,si],(4,5)),'o-')
    plt.ylim((ymin,ymax))
    plt.xlabel('Coherence')
    
    plt.subplot(224)
    plt.plot(np.reshape(temp,(4,5)),'o-')
    plt.ylim((ymin,ymax))
    plt.xlabel('Coherence')